CS20SI

Example 1

Operations
Basic operations, constants, variables 
Control dependencies 
Feeding inputs 
TensorBoard 

In [2]:
import tensorflow as tf

a = tf.constant(2)
b = tf.constant(3)
x = tf.add(a, b)
logs_dir = './graphs'

with tf.Session() as sess:
    writer = tf.summary.FileWriter(logs_dir, sess.graph)
    print(sess.run(x))
    
# close the writer when you’re done using it
#writer.close()

5


In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

# Example 1: feed_dict with placeholder
# create a placeholder of type float 32-bit, value is a vector of 3 elements
a = tf.placeholder(tf.float32, shape=[3])

# create a constant of type float 32-bit, value is a vector of 3 elements
b = tf.constant([5, 5, 5], tf.float32)

# use the placeholder as you would a constant
c = a + b  # short for tf.add(a, b)

with tf.Session() as sess:
	# print(sess.run(c)) # InvalidArgumentError because a doesn’t have any value

	# feed [1, 2, 3] to placeholder a via the dict {a: [1, 2, 3]}
	# fetch value of c
	print(sess.run(c, {a: [1, 2, 3]})) # >> [6. 7. 8.]

[ 6.  7.  8.]


In [4]:
# Example 2: feed_dict with variables
a = tf.add(2, 5)
b = tf.multiply(a, 3)

with tf.Session() as sess:
	# define a dictionary that says to replace the value of 'a' with 15
	replace_dict = {a: 15}

	# Run the session, passing in 'replace_dict' as the value to 'feed_dict'
	print(sess.run(b, feed_dict=replace_dict)) # >> 45

45


# Example 02 lazy_loading.py

When you create a Variable you pass a Tensor as its initial value to the Variable() constructor. TensorFlow provides a collection of ops that produce tensors often used for initialization from constants or random values.

Note that all these ops require you to specify the shape of the tensors. That shape automatically becomes the shape of the variable. Variables generally have a fixed shape, but TensorFlow provides advanced mechanisms to reshape variables.

# Create two variables.
weights = tf.Variable(tf.random_normal([784, 200], stddev=0.35),
                      name="weights")
biases = tf.Variable(tf.zeros([200]), name="biases")
Calling tf.Variable() adds several ops to the graph:

A variable op that holds the variable value.
An initializer op that sets the variable to its initial value. This is actually a tf.assign op.
The ops for the initial value, such as the zeros op for the biases variable in the example are also added to the graph.
The value returned by tf.Variable() value is an instance of the Python class tf.Variable.

https://www.tensorflow.org/programmers_guide/variables

# Inigializing variables

Before you can use a variable, it must be initialized. If you are programming in the low-level TensorFlow API (that is, you are explicitly creating your own graphs and sessions), you must explicitly initialize the variables. Most high-level frameworks such as tf.contrib.slim, tf.estimator.Estimator and Keras automatically initialize variables for you before training a model.

Explicit initialization is otherwise useful because it allows you not to rerun potentially expensive initializers when reloading a model from a checkpoint as well as allowing determinism when randomly-initialized variables are shared in a distributed setting.

To initialize all trainable variables in one go, before training starts, call tf.global_variables_initializer(). This function returns a single operation responsible for initializing all variables in the tf.GraphKeys.GLOBAL_VARIABLES collection. Running this operation initializes all variables. For example:

session.run(tf.global_variables_initializer())
# Now all variables are initialized.
If you do need to initialize variables yourself, you can run the variable's initializer operation. For example:

session.run(my_variable.initializer)

In [5]:
######################################## 
## NORMAL LOADING   			      ##
## print out a graph with 1 Add node  ## 
########################################

x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')
z = tf.add(x, y)

with tf.Session() as sess:
	sess.run(tf.global_variables_initializer())
	writer = tf.summary.FileWriter('./graphs/l2', sess.graph)
	for _ in range(10):
		sess.run(z)
	print(tf.get_default_graph().as_graph_def())
	writer.close()

######################################## 
## LAZY LOADING   					  ##
## print out a graph with 10 Add nodes## 
########################################

x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')

with tf.Session() as sess:
	sess.run(tf.global_variables_initializer())
	writer = tf.summary.FileWriter('./graphs/l2', sess.graph)
	for _ in range(10):
		sess.run(tf.add(x, y))
	print(tf.get_default_graph().as_graph_def()) 
	writer.close()

node {
  name: "Const"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 2
      }
    }
  }
}
node {
  name: "Const_1"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Add"
  op: "Add"
  input: "Const"
  input: "Const_1"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Placeholder"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 3
        }
      }
    }
  }
}
node {
  name: "Const_2"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
   

In [8]:
import numpy as np


a = tf.constant(2)
b = tf.constant(3)
x = tf.add(a, b)
with tf.Session() as sess:
	writer = tf.summary.FileWriter('./graphs', sess.graph) 
	print(sess.run(x))
writer.close() # close the writer when you’re done using it

5


In [9]:
a = tf.constant([2, 2], name='a')
b = tf.constant([[0, 1], [2, 3]], name='b')
x = tf.multiply(a, b, name='dot_product')
with tf.Session() as sess:
	print(sess.run(x))
# >> [[0 2]
#	 [4 6]]

[[0 2]
 [4 6]]


In [12]:
shape = tf.Variable([1,1,1], name='shape')
tf.zeros(shape, dtype=tf.float32, name=None)
#creates a tensor of shape and all elements will be zeros (when ran in session)

x = tf.zeros([2, 3], tf.int32) 
y = tf.zeros_like(x, optimize=True)
#print(y)
print(tf.get_default_graph().as_graph_def())
with tf.Session() as sess:
	y = sess.run(y)

node {
  name: "Const"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 2
      }
    }
  }
}
node {
  name: "Const_1"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 3
      }
    }
  }
}
node {
  name: "Add"
  op: "Add"
  input: "Const"
  input: "Const_1"
  attr {
    key: "T"
    value {
      type: DT_INT32
    }
  }
}
node {
  name: "Placeholder"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 3
        }
      }
    }
  }
}
node {
  name: "Const_2"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
   

In [13]:
with tf.Session() as sess:
	print(sess.run(tf.linspace(10.0, 13.0, 4)))
	print(sess.run(tf.range(5)))
	for i in np.arange(5):
		print(i)

[ 10.  11.  12.  13.]
[0 1 2 3 4]
0
1
2
3
4


In [14]:
samples = tf.multinomial(tf.constant([[1., 3., 1]]), 5)

with tf.Session() as sess:
	for _ in range(10):
		print(sess.run(samples))

[[1 1 1 1 1]]
[[1 1 1 1 1]]
[[1 1 2 1 1]]
[[1 1 1 1 1]]
[[1 2 1 2 1]]
[[1 1 1 2 1]]
[[1 2 1 1 0]]
[[1 1 1 0 1]]
[[1 1 1 1 1]]
[[1 2 1 1 1]]


In [15]:
t_0 = 19 
x = tf.zeros_like(t_0) # ==> 0
y = tf.ones_like(t_0) # ==> 1

with tf.Session() as sess:
	print(sess.run([x, y]))

[0, 1]


In [16]:
t_1 = ['apple', 'peach', 'banana']
x = tf.zeros_like(t_1) # ==> ['' '' '']
y = tf.ones_like(t_1) # ==> TypeError: Expected string, got 1 of type 'int' instead.

TypeError: Expected string, got 1 of type 'int' instead.

In [17]:
t_2 = [[True, False, False],
       [False, False, True],
       [False, True, False]] 
x = tf.zeros_like(t_2) # ==> 2x2 tensor, all elements are False
y = tf.ones_like(t_2) # ==> 2x2 tensor, all elements are True
with tf.Session() as sess:
	print(sess.run([x, y]))

[array([[False, False, False],
       [False, False, False],
       [False, False, False]], dtype=bool), array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]], dtype=bool)]


In [19]:
with tf.variable_scope('meh') as scope:
	a = tf.get_variable('a', [10])
	b = tf.get_variable('b', [100])

writer = tf.summary.FileWriter('test', tf.get_default_graph())

ValueError: Variable meh/a already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-18-d59385f0084e>", line 2, in <module>
    a = tf.get_variable('a', [10])
  File "/home/niu/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/niu/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2850, in run_ast_nodes
    if self.run_code(code, result):


In [31]:
x = tf.Variable(2.0)
y = 2.0 * (x ** 3) #** Exponent  dy = 6x^2
z = 3.0 + y ** 2 #dz = 2y = 24x^5
grad_z = tf.gradients(z, [x, y])
with tf.Session() as sess:
	sess.run(x.initializer)
	print(sess.run(grad_z))

[768.0, 32.0]


In [32]:
# Example 1: how to run assign op
W = tf.Variable(10)
assign_op = W.assign(100)

with tf.Session() as sess:
	sess.run(W.initializer)
	print(W.eval()) # >> 10
	print(sess.run(assign_op)) # >> 100

10
100


In [33]:
# Example 2: tricky example
# create a variable whose original value is 2
my_var = tf.Variable(2, name="my_var") 

# assign 2 * my_var to my_var and run the op my_var_times_two
my_var_times_two = my_var.assign(2 * my_var)

with tf.Session() as sess:
	sess.run(tf.global_variables_initializer())
	print(sess.run(my_var_times_two)) # >> 4
	print(sess.run(my_var_times_two)) # >> 8
	print(sess.run(my_var_times_two)) # >> 16

4
8
16


In [34]:
# Example 3: each session maintains its own copy of variables
W = tf.Variable(10)
sess1 = tf.Session()
sess2 = tf.Session()

# You have to initialize W at each session
sess1.run(W.initializer)
sess2.run(W.initializer)